In [68]:
# auther: JAMES LIAO 廖品捷
# copyright
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import os
import threading
import ssl
import scrapy

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'
}


In [85]:
def MultiThread_Crawl(url):
    try:
        return requests.get(url, headers = headers)
    except:
        pass

In [17]:
def GetNews_chinatimes(response):
    soup = BeautifulSoup(response.text)
    url = soup.find('link')['href']
    ndf = pd.DataFrame(data = [{'TITLE':soup.find('h1', attrs={'class':'article-title'}).text,
                                'TIME':datetime.strptime(soup.find('meta', attrs={'property':'article:published_time'})['content'],'%Y-%m-%dT%H:%M:%S+08:00'),
                                'CATEGORY':soup.find('meta',attrs={'property':'article:section'})['content'],
                                'DESCRIPTION':soup.find('meta',attrs={'name':'description'})['content'],
                                'CONTENT':'\n'.join(i.text for i in soup.find('div',attrs={'class':'article-body'}).find_all('p')),
                                'KEYWORDS':soup.find('meta',{'name':'keywords'})['content'],
                                'FROM':soup.find('meta',{'name':'publisher'})['content'],
                                'LINK':soup.find('meta', {'property':'og:url'})['content']}],
                       columns = ['TITLE', 'TIME', 'CATEGORY', 'DESCRIPTION', 'CONTENT','KEYWORDS', 'FROM', 'LINK']) 
    return ndf

In [148]:
def GetNews_ltn(response):
    soup = BeautifulSoup(response.text)
    url = soup.find('link')['href']
    ndf = pd.DataFrame(data = [{'TITLE':soup.find('h1').text,
                                'TIME':datetime.strptime(soup.find('meta', attrs={'property':'article:published_time'})['content'],'%Y-%m-%dT%H:%M:%S+08:00'),
                                'CATEGORY':soup.find('meta',attrs={'property':'article:section'})['content'],
                                'DESCRIPTION':soup.find('meta',attrs={'name':'description'})['content'],
                                'CONTENT':'\n'.join(i.text for i in soup.find('div',attrs={'class':'text boxTitle boxText'}).find_all('p')),
                                'KEYWORDS':soup.find('meta',{'name':'keywords'})['content'],
                                'FROM':soup.find('meta',{'name':'author'})['content'],
                                'LINK':soup.find('meta', {'property':'og:url'})['content']}],
                       columns = ['TITLE', 'TIME', 'CATEGORY', 'DESCRIPTION', 'CONTENT','KEYWORDS', 'FROM', 'LINK']) 
    return ndf

In [145]:
def GetLinks_chinatimes(response):
    links = []
    soup = BeautifulSoup(response.text)
    for i in soup.find_all('h3'):
        print(i)
        url = i.find('a')['href']
        links.append(url)
    return links

def GetLinks_ltn(response):
    links = []
    soup = BeautifulSoup(response.text)
    for i in soup.find_all("div", {"class": "cont"}):
        url = i['href']
        links.append(url)
    return links

In [146]:
url = 'https://news.ltn.com.tw/search/?keyword=covid' #'https://news.ltn.com.tw/search/?keyword=反紅媒'
resp = requests.get(url)
res = GetLinks_ltn(resp)
print(res)

['https://news.ltn.com.tw/news/life/breakingnews/3811896', 'https://news.ltn.com.tw/news/society/breakingnews/3811887', 'https://news.ltn.com.tw/news/life/breakingnews/3811895', 'https://news.ltn.com.tw/news/life/breakingnews/3811882', 'https://news.ltn.com.tw/news/life/breakingnews/3811880', 'https://news.ltn.com.tw/news/life/breakingnews/3811838', 'https://news.ltn.com.tw/news/life/breakingnews/3811849', 'https://news.ltn.com.tw/news/society/breakingnews/3811848', 'https://news.ltn.com.tw/news/life/breakingnews/3811769', 'https://ec.ltn.com.tw/article/breakingnews/3811844', 'https://news.ltn.com.tw/news/life/breakingnews/3811822', 'https://news.ltn.com.tw/news/world/breakingnews/3811789', 'https://news.ltn.com.tw/news/world/breakingnews/3811747', 'https://news.ltn.com.tw/news/world/breakingnews/3811519', 'https://health.ltn.com.tw/article/breakingnews/3811790', 'https://news.ltn.com.tw/news/life/breakingnews/3811739', 'https://news.ltn.com.tw/news/world/breakingnews/3811635', 'https:

In [147]:
res = MultiThread_Crawl('https://news.ltn.com.tw/news/life/breakingnews/3811896')
GetNews_LTN(res)

,TITLE,TIME,CATEGORY,DESCRIPTION,CONTENT,KEYWORDS,FROM,LINK
0,謠言終結站》網傳WHO舉白旗、承認疫苗無作用？ 錯！,2022-01-24 23:48:17,life,網路上近日有文章指稱，世界衛生組織（WHO）對武漢肺炎（新型冠狀病毒病，COVID-19）疫...,網傳文章宣稱WHO認為疫苗已經沒有作用，事實上是錯誤解讀WHO的臨時聲明。（法新社）\n〔即...,"世界衛生組織,疫苗,世衛組織,武漢肺炎,新型冠狀病毒,COVID-19,新型冠狀病毒病",自由時報電子報,https://news.ltn.com.tw/news/life/breakingnews...


In [153]:
def Search_chinatimes(keywords, pages):
    # crawling
    links = []
    prev = 0
    for i in range(pages):
        url = 'https://www.chinatimes.com/search/{}?page={}'.format(keywords, i+1)
        resp = requests.get(url)
        links += GetLinks_chinatimes(resp)
        links = list(set(links))  
        page_n = len(links)
        print('There are {} links in page {} | total {}'.format(page_n - prev,str(i + 1), page_n))
        prev = page_n

    # 多線程爬蟲

    responses = [MultiThread_Crawl(link) for link in links]

    # 整理成DataFrame
    list_of_dataframes = []
    for response in responses:
        try:
            ndf = GetNews_chinatimes(response)
            list_of_dataframes.append(ndf)
        except:
            pass
    df = pd.concat(list_of_dataframes, ignore_index=True)
    print('There are {} News in DataFrame.'.format(len(df)))
    return df


def Search_ltn(keywords, pages):
    # crawling
    # https://search.ltn.com.tw/list?keyword=covid&start_time=20041201&end_time=20220125&sort=date&type=all&page=2
    links = []
    prev = 0
    for i in range(pages):
        url = 'https://search.ltn.com.tw/list?keyword={}&page={}'.format(keywords, i+1)
        resp = requests.get(url)
        links += GetLinks_ltn(resp)
        links = list(set(links))  
        page_n = len(links)
        print('There are {} links in page {} | total {}'.format(page_n - prev,str(i + 1), page_n))
        prev = page_n

    # 多線程爬蟲

    responses = [MultiThread_Crawl(link) for link in links]

    # 整理成DataFrame
    list_of_dataframes = []
    for response in responses:
        try:
            ndf = GetNews_ltn(response)
            list_of_dataframes.append(ndf)
        except:
            pass
    df = pd.concat(list_of_dataframes, ignore_index=True)
    print('There are {} News in DataFrame.'.format(len(df)))
    return df




In [154]:
%%time
# 可以自行替換查詢的關鍵字，另外需要更多新聞的人也可以把 pages 的數值調高)
df = Search_ltn(keywords='covid', pages=5)

There are 20 links in page 1 | total 20
There are 20 links in page 2 | total 40
There are 20 links in page 3 | total 60
There are 20 links in page 4 | total 80
There are 20 links in page 5 | total 100
There are 78 News in DataFrame.
CPU times: user 5.19 s, sys: 163 ms, total: 5.36 s
Wall time: 58.4 s


In [155]:
df.tail()

,TITLE,TIME,CATEGORY,DESCRIPTION,CONTENT,KEYWORDS,FROM,LINK
73,鹿港鎮宣布停辦春節與元宵踩街 小提燈取消現場發放,2022-01-24 14:31:09,life,面對武漢肺炎（新型冠狀病毒病，COVID-19）疫情升溫，彰化縣鹿港鎮長許志宏宣布，今年新春...,有「台灣版萬聖節」之稱的鹿港頂番婆鬧元宵活動，去年、前年都照常舉行。（資料照，記者劉曉欣攝）...,武漢肺炎,自由時報電子報,https://news.ltn.com.tw/news/life/breakingnews...
74,苗栗縣原住民族假日藝文展演啟動 到年底展演133場次,2022-01-24 16:29:06,life,苗栗縣原住民族假日藝文展演活動，從2月1日至12月底在泰雅文物館與賽夏族民俗文物館兩館舉辦1...,傳源文化藝術團表演泰雅族歌舞。（記者張勳騰攝）\n〔記者張勳騰／苗栗報導〕苗栗縣原住民族假日...,"原住民族,原住民",自由時報電子報,https://news.ltn.com.tw/news/life/breakingnews...
75,旗津擴大篩檢 陳其邁：拜託大家篩一篩、過年較安心,2022-01-24 16:08:25,life,鑒於高雄港群聚個案不少足跡在旗津，高雄市政府即日起擴大旗津社區篩檢，市長陳其邁今也向旗津居民...,陳其邁呼籲旗津鄉親與工作者，儘快接受快篩。（記者李惠洲攝）\n\n〔記者王榮祥／高雄報導〕鑒...,"高雄港,旗津,武漢肺炎,新型冠狀病毒,COVID-19,新型冠狀病毒病",自由時報電子報,https://news.ltn.com.tw/news/life/breakingnews...
76,雲林春節期間接種不打烊 合約院所拚打疫苗,2022-01-24 21:16:57,life,雲林縣政府春節期間疫苗保護不打烊，縣府經與合約院所協調，於春節連假期間都有醫療院服提供接種服...,雲林春節疫苗接種不打烊，合約院所繼續為鄉親施打疫苗。（記者詹士弘翻攝）\n〔記者詹士弘／雲林...,"武漢肺炎,新型冠狀病毒,COVID-19,新型冠狀病毒病",自由時報電子報,https://news.ltn.com.tw/news/life/breakingnews...
77,阻絕疫情於境外 楊鎮浯：強力建議中央台灣端機場設快篩站,2022-01-24 22:55:52,life,金門縣長楊鎮浯今天在金門縣流行疫情指揮中心主持今年第1次會議後指出，已與澎湖、連江縣取得共識...,金門縣長楊鎮浯（左起）、縣府秘書長陳祥麟、縣衛生局長李錫鑫三人於地區首次防疫會議前召開「會前...,"機場,快篩,武漢肺炎,新型冠狀病毒,COVID-19,新型冠狀病毒病",自由時報電子報,https://news.ltn.com.tw/news/life/breakingnews...
